In [1]:
pip install anthropic

Note: you may need to restart the kernel to use updated packages.


In [1]:
from anthropic import Anthropic

MODEL_NAME="claude-3-5-sonnet-20241022"
client = Anthropic(
    api_key="<enter-your-api-key>",
)

## Prompt

In [35]:
# 역할 부여
setting_the_role = """
You are an advanced AI assistant tasked with creating optimal class schedules for students based on a provided list of classes. 
Your goal is to generate one or more schedule cases that prioritize certain classes while adhering to specific rules and constraints.

First, here is the list of classes you'll be working with:

<class_list>
{{CLASS_LIST}}
</class_list>

Please follow these steps to create the class schedule(s):
"""

In [36]:
instruction_pt1 = """
1. The input will contain a structured list of courses categorized by their type, credit hours, 
and any relevant details regarding their scheduling. 
Each line in the class list contains the following information in order: title, professor, day of week, time, category, and credit.
   
class list: 컴퓨터공학개론, 전혜경, 월요일 2-3, 전공기초, 2
-> title: 컴퓨터공학개론
-> professor: 전혜경
-> day of week: 월요일
-> time: 2-3
-> category: 전공기초
-> credit: 2
"""

In [45]:
instruction_pt2 = """
2. Create schedule cases following these rules:
   a. [important] Don't overlap any classes.
       Identify any overloaded days and suggest balancing alternatives.
       Propose alternative course selections or different cases to avoid conflicts.
   b. A categorized list of courses including:
        - 전공핵심 (Core Major): Courses directly related to the major that are essential for advanced studies.
        - 전공기초 (Foundation Major): Fundamental courses that build foundational knowledge in the field.
        - 교양과목 (General Education): A mix of essential liberal arts and elective courses.
   c. Use the following time representation:
      - 9:00-10:00 -> 1
      - 10:00-11:00 -> 2
      - 11:00-12:00 -> 3
      - 12:00-13:00 -> 4
      - 13:00-14:00 -> 5
      - 14:00-15:00 -> 6
      - 15:00-16:00 -> 7
      - 16:00-17:00 -> 8
      - 17:00-18:00 -> 9
   d. Use 'A' for 15 minutes and 'B' for 30 minutes:
      - 1-2A means 9:00-10:15
      - 2B-3 means 10:30-12:00
      - 8B-9 means 16:30-18:00
   d. [important] Generate at least two different course schedules to accommodate flexibility.
       If there's conflict, You must list the schedules of all possible cases!!! 
   e. If classes share the same time slot, create one more case of schedule.
"""

In [47]:
instruction_pt3 = """
3. Calculate sum of credits for each schedule case and write into <total_score> tags.
Credit Limit Compliance: Ensure the total credits do not exceed 21.
"""

In [49]:
instruction_pt4 = """
4. Present your output in the following format.
    Each case should include a structured timetable in a tabular format:

<schedule_case>
<case_number>1</case_number>
<timetable>
| 시간  | 월요일 | 화요일 | 수요일 | 목요일 | 금요일 |
|------|--------|--------|--------|--------|--------|
| 1    |        |        |        |        |        |
|       |        |        |        |        |        |
| 2    |        |        |        |        |        |
|       |        |        |        |        |        |
| 3    |        |        |        |        |        |
|       |        |        |        |        |        |
| 4    |        |        |        |        |        |
|       |        |        |        |        |        |
| 5    |        |        |        |        |        |
|       |        |        |        |        |        |
| 6    |        |        |        |        |        |
|       |        |        |        |        |        |
| 7    |        |        |        |        |        |
|       |        |        |        |        |        |
| 8    |        |        |        |        |        |
|       |        |        |        |        |        |
| 9    |        |        |        |        |        |
|       |        |        |        |        |        |
</timetable>
<total_score>X</total_score>
</schedule_case>
"""

In [50]:
instruction_pt5 = """
5. In the timetable, fill each cell with the following information:
   - Class title
   - Professor's name
   - Exact class time
   Leave empty cells blank.
"""

In [51]:
instruction_pt6 = """
6. A Detailed Explanation in Korean (not english)
    - Justification for the proposed schedules.
    - Notes on alternative choices and optimizations.
    - Recommendations for handling back-to-back classes or high-credit loads.

provide a brief explanation of your scheduling process and any trade-offs you made in creating the schedule(s) within <explanation> tags.

"""

In [52]:
instruction_pt7 = """
Before you begin creating the schedule(s), please use <analysis> tags to:
1. List all classes, categorizing them by priority (Core major, Foundation major, Other).
2. Count the number of classes in each category.
3. Identify any potential scheduling conflicts or challenges.
4. Outline a high-level plan for creating the schedule(s).
"""

In [53]:
instruction_pt8 = """
<example>
<analysis>
1. Class Categories:

Core Major (전공핵심):
	•Data Structures (3 credits)
	•Data Programming (3 credits)

Foundation Major (전공기초):
	•Introduction to Computer Engineering (2 credits)
	•Introduction to Information Security (2 credits)
	•Discrete Mathematics (3 credits)

Other (기타):
	•Presentation and Debate (Core Liberal Arts, 3 credits)
	•College English Conversation A (Basic Liberal Arts, 1 credit)
	•Writing Theory and Practice (Basic Liberal Arts, 2 credits)
	•Human Relations and Leadership (Core Liberal Arts, 3 credits)
	•Humanity and the Universe (Advanced Liberal Arts, 3 credits)
	•Startup & Social Venture (Advanced Liberal Arts, 3 credits)

2. Course Count by Category:
	•Core Major Courses: 2 courses
	•Foundation Major Courses: 3 courses
	•Liberal Arts Courses: 6 courses

3. Scheduling Conflicts:
	•Monday (4-6 PM): Human Relations and Leadership conflicts with Introduction to Information Security.
	•Friday (4-6 PM): Humanity and the Universe conflicts with Startup & Social Venture.
	•Some courses occupy consecutive time slots, requiring careful schedule adjustments.

4. Planning Approach:
	•Prioritize Core Major and Foundation Major courses.
	•Separate conflicting courses into different scheduling scenarios.
	•Ensure that the total credits do not exceed 21.
	•Create at least two alternative schedule cases for flexibility.

<schedule_case>
<case1>
<timetable> creating table </timetable>
<total_score>X</total_score>
</case1>

<case2>
<timetable> creating table </timetable>
<total_score>X</total_score>
</case2>

<case3>
<timetable> creating table </timetable>
<total_score>X</total_score>
</case3>
</schedule_case>

<explanation></explanation>

</example>
"""

In [55]:
final_prompt = f"""
{setting_the_role}
{instruction_pt1}
{instruction_pt2}
{instruction_pt3}
{instruction_pt4}
{instruction_pt5}
{instruction_pt6}
{instruction_pt7}
{instruction_pt8}
"""

print(final_prompt)



You are an advanced AI assistant tasked with creating optimal class schedules for students based on a provided list of classes. 
Your goal is to generate one or more schedule cases that prioritize certain classes while adhering to specific rules and constraints.

First, here is the list of classes you'll be working with:

<class_list>
{{CLASS_LIST}}
</class_list>

Please follow these steps to create the class schedule(s):


1. The input will contain a structured list of courses categorized by their type, credit hours, 
and any relevant details regarding their scheduling. 
Each line in the class list contains the following information in order: title, professor, day of week, time, category, and credit.
   
class list: 컴퓨터공학개론, 전혜경, 월요일 2-3, 전공기초, 2
-> title: 컴퓨터공학개론
-> professor: 전혜경
-> day of week: 월요일
-> time: 2-3
-> category: 전공기초
-> credit: 2


2. Create schedule cases following these rules:
   a. [important] Don't overlap any classes.
       Identify any overloaded days and sugge

In [68]:
def get_schedule(class_list):
    prompt = final_prompt.replace("{{CLASS_LIST}}", class_list)
    
    with client.messages.stream(
        max_tokens=3000,
        messages=[{"role": "user", "content": prompt}],
        model=MODEL_NAME,
    ) as stream:
        buffer = ""
        word_count = 0
        words_per_chunk = 5

        for text in stream.text_stream:
            buffer += text

            if " " in text:
                word_count += 1

            if word_count >= words_per_chunk:
                print(buffer, end="", flush=True)
                buffer = ""
                word_count = 0
        if buffer:
            print(buffer, flush=True)

In [69]:
input_list = """
<class_list>
컴퓨터공학개론, 전혜경, 월요일 2-3, 전공기초, 2
정보보안개론, 민병준, 월요일 5-6, 전공기초, 2
자료구조, 박성호, 화요일 1-2A & 목요일 8B-9, 전공핵심, 3
데이터프로그래밍, 이장호, 화요일 2B-3 & 목요일 1-2A, 전공핵심, 3
발표와토론, 안호림, 수요일 2-4, 핵심교양, 3
대학영어회화, 브루스, 목요일 6-7, 기초교양, 1
이산수학, 김지범, 수요일 5B-6, 금요일 1-2A, 전공기초, 3
글쓰기이론과실제, 정영진, 화요일 4-6, 기초교양, 2
인간관계와리더십, 김재득, 월요일 4-6, 핵심교양, 3
인간과우주, 정연철, 금요일 4-6, 심화교양, 3
스타트업소셜벤처, 창업지원단, 금4-6, 심화교양, 3
</class_list>
"""

In [70]:
get_schedule(input_list)

I'll help create optimal class schedules based on the provided information. Let me start with the analysis.

<analysis>
1. 수업 카테고리:

전공핵심 (Core Major):
• 자료구조 (3학점) - 화1-2A & 목8B-9
• 데이터프로그래밍 (3학점) - 화2B-3 & 목1-2A

전공기초 (Foundation Major):
• 컴퓨터공학개론 (2학점) - 월2-3
• 정보보안개론 (2학점) - 월5-6
• 이산수학 (3학점) - 수5B-6 & 금1-2A

교양과목:
기초교양:
• 대학영어회화 (1학점) - 목6-7
• 글쓰기이론과실제 (2학점) - 화4-6

핵심교양:
• 발표와토론 (3학점) - 수2-4
• 인간관계와리더십 (3학점) - 월4-6

심화교양:
• 인간과우주 (3학점) - 금4-6
• 스타트업소셜벤처 (3학점) - 금4-6

2. 과목 수:
• 전공핵심: 2과목
• 전공기초: 3과목
• 교양과목: 6과목

3. 시간 충돌:
• 월요일 4-6교시: 정보보안개론과 인간관계와리더십 충돌
• 금요일 4-6교시: 인간과우주와 스타트업소셜벤처 충돌
• 화요일: 자료구조와 데이터프로그래밍 연계 시간 주의 필요

4. 계획 접근방식:
• 전공핵심/기초 과목 우선 배치
• 충돌하는 과목들은 별도 케이스로 분리
• 총 학점 21학점 초과하지 않도록 조정
• 최소 2개 이상의 대안 스케줄 생성
</analysis>

[스케줄 케이스 생성 진행]

<schedule_case>
<case_number>1</case_number>
<timetable>
| 시간 | 월요일 | 화요일 | 수요일 | 목요일 | 금요일 |
|------|--------|--------|--------|--------|--------|
| 1    |        |자료구조|        |데이터프로그래밍|이산수학|
|      |        |박성호  |        |이장호  |김지범 